Installing pyDOE

In [ ]:
!pip install pyDOE

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pyDOE: filename=pyDOE-0.3.8-py3-none-any.whl size=18184 sha256=8f46b139aa1e071b7020fc33b8b60afed4f3de8b6f576f58f57e2a5409574371
  Stored in directory: /root/.cache/pip/wheels/83/ce/8a/87b25c685bfeca1872d13b8dc101e087a9c6e3fb5ebb47022a
Successfully built pyDOE


Importing libraries

In [ ]:
import os
import random
import math

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow.keras.layers as L
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import tensorflow_hub as hub
import numpy as np
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from tabulate import tabulate
from pyDOE import lhs
import matplotlib.pyplot as plt

Importing the Training Dataset

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      rescale = 1./255,
      rotation_range=40,
      horizontal_flip=True,
      width_shift_range=0.2, 
      height_shift_range=0.2,
      shear_range=0.2, 
      zoom_range=0.2,
      fill_mode='nearest' )
train_generator = train_datagen.flow_from_directory(
    "./drive/MyDrive/RiceLeafDiseaseDataSet/train/", 
    subset="training", 
    shuffle=True, 
    seed=42,
    color_mode="rgb", 
    class_mode="categorical",
    target_size=(256,256),
    batch_size=64)

Found 1216 images belonging to 4 classes.


In [ ]:
train_generator.class_indices

{'Bacterial leaf blight': 0, 'Brown spot': 1, 'Healthy': 2, 'Leaf smut': 3}

Importing the validation Dataset

In [ ]:
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    "./drive/MyDrive/RiceLeafDiseaseDataSet/test/", 
    shuffle=False, 
    seed=42,
    color_mode="rgb", 
    class_mode="categorical",
    target_size=(256,256),
    batch_size=64)


Found 205 images belonging to 4 classes.


In [ ]:
validation_generator.class_indices

{'Bacterial leaf blight': 0, 'Brown spot': 1, 'Healthy': 2, 'Leaf smut': 3}

Loading the Feature Vector Extraction Model

In [ ]:
embed =hub.KerasLayer('https://tfhub.dev/google/bit/m-r50x1/1',trainable=True)

Creating the Model

In [ ]:
model = tf.keras.Sequential([
    embed,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(4, activation='softmax',
                           kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])
model.build((None,)+(256,256)+(3,))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 2048)              23500352  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 4)                 2052      
                                                                 
Total params: 24,551,492
Trainable params: 24,551,492
Non-trainable params: 0
_________________________________________________________________


Compiling the Model

In [ ]:
#Compile model specifying the optimizer learning rate

LEARNING_RATE = 0.001 #@param {type:"number"}

model.compile(
   optimizer=tf.keras.optimizers.Adam(lr=LEARNING_RATE), 
   loss='categorical_crossentropy',
   metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Training the Model

In [ ]:
EPOCHS=30
model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples//train_generator.batch_size,
        epochs=EPOCHS,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples//validation_generator.batch_size)

 5/19 [======>.......................] - ETA: 39s - loss: 0.0872 - accuracy: 0.9688

KeyboardInterrupt: ignored

Saving the Model

In [ ]:
model.save("/content/drive/MyDrive/Project_H5Final1571.h5",save_format="h5")
model.save("/content/drive/MyDrive/Project_H5Final1Saved1571")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Project_H5Final1Saved1571/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Project_H5Final1Saved1571/assets
